In [52]:
import pandas as pd
import numpy as np
import yfinance as yf
import time
import telegram
# from tabulate import tabulate
import matplotlib.pyplot as plt
from datetime import date
from dateutil.relativedelta import relativedelta
import asyncio
#import talib as TA

In [53]:
# Class declaration section
class Stock:
  def __init__(self, stock_code, ticker_no, close_price, close_price_ld, close_price_l5d):
    self.stock_code,  self.ticker_no = stock_code, ticker_no
    self.close_price, self.close_price_ld = to_float(close_price), to_float(close_price_ld)
    self.close_price_l5d = to_float(close_price_l5d)
    self.close_price_diff = self.close_price - self.close_price_ld
    self.daily_return = self.calc_return(self.close_price, self.close_price_ld)
    self.five_days_return = self.calc_return(self.close_price, self.close_price_l5d)
    
  def calc_return(self, close_price, close_price_ld):
    return (to_float(100*(close_price-close_price_ld)/close_price_ld))

In [54]:
# function declaration section
def to_ticker(stock_code):
    stock_code = '0000' + stock_code
    ticker_no = stock_code[-4:] + '.HK'
    return(ticker_no)

def to_float(s):
    return(round(float(s), 4 ))

def get_stock_data(stock_code, ticker_no, init_date):
    df_stock = yf.download(ticker_no, init_date, progress=False)
    close_price = df_stock.iloc[-1]["Close"]
    close_price_ld = df_stock.iloc[-2]["Close"]
    close_price_l5d = df_stock.iloc[-5]["Close"]
    
    close_price_date = df_stock.index.values[-1]
    close_price_ld_date = df_stock.index.values[-2]
    close_price_l5d_date = df_stock.index.values[-5]
    print(close_price_date)
    print(close_price_ld_date)
    print(close_price_l5d_date)
    
    stock = Stock(stock_code, ticker_no, close_price, close_price_ld, close_price_l5d)
    return(stock)

In [55]:
# init
today=date.today()
today_diff=today - relativedelta(months=1)
today_before=today_diff.isoformat()
today=today.isoformat()

service = Service()
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=service, options=options)

In [56]:
# import excel file
df_stock_list_xlsx=pd.read_excel("C:/temp/My_Stock_monitoring_list_1.xlsx")
df_stock_list_xlsx = df_stock_list_xlsx.astype(str)  # convert all columns to string type
ticker_list = [to_ticker(stock_code) for stock_code in list(df_stock_list_xlsx['Stock Code'])]

df_stock_list=df_stock_list_xlsx.copy()
df_stock_list['Ticker No']=ticker_list
df_stock_list.loc[len(df_stock_list.index)]=['HSI','^HSI']
df_stock_list

,Stock Code,Ticker No
0,939,0939.HK
1,941,0941.HK
2,1398,1398.HK
3,2600,2600.HK
4,2628,2628.HK
5,HSI,^HSI


In [57]:
stocks = []
for index, row in df_stock_list.iterrows():  
    stock_code = row["Stock Code"]
    ticker_no = row["Ticker No"]
    print("Processing stock: " + ticker_no + "....")
    stock = get_stock_data(stock_code, ticker_no, today_before)
    stocks.append(stock)
print("Processing Ended")
    
df_stock_fin_list=pd.DataFrame([stock.__dict__ for stock in stocks])
df_stock_fin_list.columns = ['Stock Code', 'Ticker No', 'Latest Price', 'Last Day Price', 'Last 5 Day Price', 'Price Difference', 'Daily Return %', '5 Day Return %' ]
df_stock_fin_list

Processing stock: 0939.HK....
2023-08-03T00:00:00.000000000
2023-08-02T00:00:00.000000000
2023-07-28T00:00:00.000000000
Processing stock: 0941.HK....
2023-08-03T00:00:00.000000000
2023-08-02T00:00:00.000000000
2023-07-28T00:00:00.000000000
Processing stock: 1398.HK....
2023-08-03T00:00:00.000000000
2023-08-02T00:00:00.000000000
2023-07-28T00:00:00.000000000
Processing stock: 2600.HK....
2023-08-03T00:00:00.000000000
2023-08-02T00:00:00.000000000
2023-07-28T00:00:00.000000000
Processing stock: 2628.HK....
2023-08-03T00:00:00.000000000
2023-08-02T00:00:00.000000000
2023-07-28T00:00:00.000000000
Processing stock: ^HSI....
2023-08-03T00:00:00.000000000
2023-08-02T00:00:00.000000000
2023-07-28T00:00:00.000000000
Processing Ended


,Stock Code,Ticker No,Latest Price,Last Day Price,Last 5 Day Price,Price Difference,Daily Return %,5 Day Return %
0,939,0939.HK,4.3300,4.3500,4.4800,-0.0200,-0.4598,-3.3482
1,941,0941.HK,63.6500,64.0500,65.0000,-0.4000,-0.6245,-2.0769
2,1398,1398.HK,3.6500,3.6700,3.7400,-0.0200,-0.5450,-2.4064
3,2600,2600.HK,3.8000,3.7900,3.7700,0.0100,0.2639,0.7958
4,2628,2628.HK,13.5200,13.3600,13.4800,0.1600,1.1976,0.2967
5,HSI,^HSI,19420.8691,19517.3809,19916.5605,-96.5118,-0.4945,-2.4888


In [58]:
list(df_stock_fin_list['Ticker No'])

['0939.HK', '0941.HK', '1398.HK', '2600.HK', '2628.HK', '^HSI']

In [59]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

import requests 
from bs4 import BeautifulSoup

In [60]:
D_tickers = {}
L_tickers_no = list(df_stock_fin_list['Stock Code'])
L_tickers = list(df_stock_fin_list['Ticker No'])
L_tickers_curr,L_tickers_updown,L_tickers_updown_per,L_tickers_name = [],[],[],[]
for aticker in L_tickers:
    print("Scraping stock: " + aticker + "....")
    url = f'https://hk.finance.yahoo.com/quote/{aticker}?p={aticker}&.tsrc=fin-srch'
    driver.get(url)
    #time.sleep(1)
    html=driver.page_source
    text_non= html.replace("\n","")
    bsObj = BeautifulSoup(text_non, "html.parser")    
    priceDiv=bsObj.find_all("div", { "class" : "D(ib) Mend(20px)"} )
    for idx, c in enumerate(priceDiv[0].children):
        if idx == 0:
            L_tickers_curr.append(c.text) 
        if idx == 1:
            L_tickers_updown.append(c.text)
        if idx == 3:
            L_tickers_updown_per.append(c.text)
    stockName=bsObj.find("h1", { "class" : "D(ib) Fz(18px)"} )    
    L_tickers_name.append(stockName.text)
D_tickers['Stock Code']=L_tickers_no
D_tickers['Stock Name']=L_tickers_name
D_tickers['Curr Price']=L_tickers_curr
D_tickers['Up Down']=L_tickers_updown
D_tickers['Up Down %']=L_tickers_updown_per
df_realtime=pd.DataFrame(D_tickers)
# df_realtime

df_result = pd.merge(df_realtime, df_stock_fin_list, on='Stock Code')
df_result[['Stock Code', 'Stock Name', 'Curr Price', 'Up Down', 'Up Down %', 'Latest Price', 'Last Day Price','Price Difference','Last 5 Day Price', 'Daily Return %', '5 Day Return %']]

Scraping stock: 0939.HK....
Scraping stock: 0941.HK....
Scraping stock: 1398.HK....
Scraping stock: 2600.HK....
Scraping stock: 2628.HK....
Scraping stock: ^HSI....


,Stock Code,Stock Name,Curr Price,Up Down,Up Down %,Latest Price,Last Day Price,Price Difference,Last 5 Day Price,Daily Return %,5 Day Return %
0,939,建設銀行 (0939.HK),4.330,-0.020,(-0.46%),4.3300,4.3500,-0.0200,4.4800,-0.4598,-3.3482
1,941,中國移動 (0941.HK),63.650,-0.400,(-0.62%),63.6500,64.0500,-0.4000,65.0000,-0.6245,-2.0769
2,1398,工商銀行 (1398.HK),3.650,-0.020,(-0.54%),3.6500,3.6700,-0.0200,3.7400,-0.5450,-2.4064
3,2600,中國鋁業 (2600.HK),3.800,+0.010,(+0.26%),3.8000,3.7900,0.0100,3.7700,0.2639,0.7958
4,2628,中國人壽 (2628.HK),13.520,+0.160,(+1.20%),13.5200,13.3600,0.1600,13.4800,1.1976,0.2967
5,HSI,恒生指數 (^HSI),"19,420.87",-96.51,(-0.49%),19420.8691,19517.3809,-96.5118,19916.5605,-0.4945,-2.4888


In [62]:
driver.quit()
# In the end, always close or quit the driver to ensure all system resources are freed up